In [ ]:
# تثبيت المكتبات
!pip install pose-format numpy pandas pyarrow

In [ ]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from pose_format import Pose

## 📥 قراءة ملف .pose أولاً

In [ ]:
def load_pose_file(pose_path: str) -> Pose:
    """
    تحميل ملف .pose
    """
    with open(pose_path, 'rb') as f:
        pose = Pose.read(f.read())
    return pose

# تحميل الملف
# pose = load_pose_file("path/to/your/file.pose")

## 1️⃣ حفظ كملف .pose (الصيغة الأصلية)

In [ ]:
def save_as_pose(pose: Pose, output_path: str):
    """
    حفظ كملف .pose
    """
    with open(output_path, 'wb') as f:
        pose.write(f)
    print(f"✅ تم الحفظ في: {output_path}")
    
# save_as_pose(pose, "output.pose")

## 2️⃣ تحويل إلى JSON

In [ ]:
def pose_to_json(pose: Pose, output_path: str = None, include_header: bool = True):
    """
    تحويل Pose إلى JSON
    
    Parameters:
    -----------
    pose : Pose
        كائن الـ Pose
    output_path : str, optional
        مسار الملف الناتج. إذا لم يُحدد، يُرجع القاموس فقط
    include_header : bool
        تضمين معلومات الـ header
    """
    # تحويل البيانات إلى قوائم Python
    data = pose.body.data.filled(0).tolist()  # استبدال القيم المفقودة بـ 0
    confidence = pose.body.confidence.tolist()
    
    result = {
        "data": data,
        "confidence": confidence,
        "fps": pose.body.fps,
        "shape": {
            "frames": pose.body.data.shape[0],
            "people": pose.body.data.shape[1],
            "points": pose.body.data.shape[2],
            "dimensions": pose.body.data.shape[3]
        }
    }
    
    if include_header:
        result["header"] = {
            "version": pose.header.version,
            "dimensions": {
                "width": pose.header.dimensions.width,
                "height": pose.header.dimensions.height,
                "depth": pose.header.dimensions.depth
            },
            "components": [
                {
                    "name": comp.name,
                    "points": comp.points,
                    "format": comp.format,
                    "limbs": comp.limbs,
                    "num_points": len(comp.points)
                }
                for comp in pose.header.components
            ]
        }
    
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2)
        print(f"✅ تم الحفظ في: {output_path}")
    
    return result

# json_data = pose_to_json(pose, "output.json")

In [ ]:
def pose_to_json_compact(pose: Pose, output_path: str):
    """
    تحويل إلى JSON مضغوط (بدون مسافات)
    """
    data = pose.body.data.filled(0).tolist()
    confidence = pose.body.confidence.tolist()
    
    result = {
        "d": data,
        "c": confidence,
        "f": pose.body.fps,
        "w": pose.header.dimensions.width,
        "h": pose.header.dimensions.height
    }
    
    with open(output_path, 'w') as f:
        json.dump(result, f, separators=(',', ':'))
    
    print(f"✅ تم الحفظ (مضغوط) في: {output_path}")

## 3️⃣ تحويل إلى NumPy (.npz)

In [ ]:
def pose_to_npz(pose: Pose, output_path: str, compressed: bool = True):
    """
    تحويل Pose إلى ملف NumPy (.npz)
    
    Parameters:
    -----------
    pose : Pose
        كائن الـ Pose
    output_path : str
        مسار الملف الناتج
    compressed : bool
        استخدام الضغط (يقلل الحجم لكن أبطأ في القراءة/الكتابة)
    """
    # استخراج البيانات
    data = np.array(pose.body.data.filled(0), dtype=np.float32)
    confidence = np.array(pose.body.confidence, dtype=np.float32)
    mask = np.array(pose.body.data.mask, dtype=bool)
    
    # معلومات إضافية
    metadata = {
        'fps': np.array([pose.body.fps]),
        'width': np.array([pose.header.dimensions.width]),
        'height': np.array([pose.header.dimensions.height]),
        'depth': np.array([pose.header.dimensions.depth]),
        'version': np.array([pose.header.version])
    }
    
    # حفظ
    save_func = np.savez_compressed if compressed else np.savez
    save_func(
        output_path,
        data=data,
        confidence=confidence,
        mask=mask,
        **metadata
    )
    
    print(f"✅ تم الحفظ في: {output_path}")
    print(f"   📦 حجم الملف: {Path(output_path).stat().st_size / 1024:.2f} KB")

# pose_to_npz(pose, "output.npz", compressed=True)

In [ ]:
def npz_to_pose(npz_path: str) -> dict:
    """
    قراءة ملف .npz وإرجاع البيانات
    """
    loaded = np.load(npz_path)
    
    result = {
        'data': loaded['data'],
        'confidence': loaded['confidence'],
        'mask': loaded['mask'],
        'fps': loaded['fps'][0],
        'width': loaded['width'][0],
        'height': loaded['height'][0],
    }
    
    print(f"✅ تم التحميل من: {npz_path}")
    print(f"   شكل البيانات: {result['data'].shape}")
    
    return result

# data_dict = npz_to_pose("output.npz")

## 4️⃣ تحويل إلى Parquet

In [ ]:
def pose_to_parquet(pose: Pose, output_path: str, component_name: str = None):
    """
    تحويل Pose إلى Parquet (مناسب للتحليلات الكبيرة)
    
    البنية:
    - frame_id: رقم الإطار
    - person_id: رقم الشخص
    - point_name: اسم النقطة
    - component: اسم المكون
    - x, y, z: الإحداثيات
    - confidence: درجة الثقة
    """
    rows = []
    
    data = pose.body.data
    confidence = pose.body.confidence
    
    frames, people, points, dims = data.shape
    
    # بناء قائمة النقاط مع أسمائها
    point_info = []
    for comp in pose.header.components:
        if component_name and comp.name != component_name:
            continue
        for point_name in comp.points:
            point_info.append((comp.name, point_name))
    
    print(f"🔄 جاري التحويل... ({frames} إطار)")
    
    for frame_idx in range(frames):
        for person_idx in range(people):
            for point_idx, (comp_name, point_name) in enumerate(point_info):
                conf = confidence[frame_idx, person_idx, point_idx]
                
                # تخطي النقاط ذات الثقة صفر
                if conf == 0:
                    continue
                
                coords = data[frame_idx, person_idx, point_idx]
                
                row = {
                    'frame_id': frame_idx,
                    'time_sec': frame_idx / pose.body.fps,
                    'person_id': person_idx,
                    'component': comp_name,
                    'point_name': point_name,
                    'point_index': point_idx,
                    'x': float(coords[0]),
                    'y': float(coords[1]),
                    'z': float(coords[2]) if dims > 2 else 0.0,
                    'confidence': float(conf)
                }
                rows.append(row)
    
    # إنشاء DataFrame وحفظه
    df = pd.DataFrame(rows)
    df.to_parquet(output_path, index=False, compression='snappy')
    
    print(f"✅ تم الحفظ في: {output_path}")
    print(f"   📊 عدد الصفوف: {len(df):,}")
    print(f"   📦 حجم الملف: {Path(output_path).stat().st_size / 1024:.2f} KB")
    
    return df

# df = pose_to_parquet(pose, "output.parquet")

In [ ]:
def pose_to_parquet_efficient(pose: Pose, output_path: str):
    """
    تحويل Pose إلى Parquet بطريقة أكثر كفاءة (صف لكل إطار)
    
    هذه الطريقة تخزن كل إطار كصف واحد مع النقاط كـ arrays
    """
    rows = []
    frames = pose.body.data.shape[0]
    
    for frame_idx in range(frames):
        row = {
            'frame_id': frame_idx,
            'time_sec': frame_idx / pose.body.fps,
            'data': pose.body.data[frame_idx].filled(0).flatten().tolist(),
            'confidence': pose.body.confidence[frame_idx].flatten().tolist()
        }
        rows.append(row)
    
    df = pd.DataFrame(rows)
    df.to_parquet(output_path, index=False)
    
    print(f"✅ تم الحفظ (الطريقة الفعالة) في: {output_path}")
    print(f"   📊 عدد الإطارات: {len(df)}")
    
    return df

## 5️⃣ تحويل إلى CSV

In [ ]:
def pose_to_csv(pose: Pose, output_path: str, component_filter: list = None):
    """
    تحويل Pose إلى CSV
    
    Parameters:
    -----------
    pose : Pose
        كائن الـ Pose
    output_path : str
        مسار الملف الناتج
    component_filter : list, optional
        قائمة بأسماء المكونات المطلوبة فقط
    """
    rows = []
    
    data = pose.body.data
    confidence = pose.body.confidence
    frames, people, _, _ = data.shape
    
    # بناء معلومات النقاط
    point_idx = 0
    point_info = []
    for comp in pose.header.components:
        if component_filter and comp.name not in component_filter:
            point_idx += len(comp.points)
            continue
        for p_name in comp.points:
            point_info.append((point_idx, comp.name, p_name))
            point_idx += 1
    
    print(f"🔄 جاري التحويل إلى CSV...")
    
    for frame_idx in range(frames):
        for person_idx in range(people):
            for p_idx, comp_name, point_name in point_info:
                conf = confidence[frame_idx, person_idx, p_idx]
                if conf == 0:
                    continue
                    
                coords = data[frame_idx, person_idx, p_idx]
                rows.append({
                    'frame': frame_idx,
                    'time': round(frame_idx / pose.body.fps, 4),
                    'person': person_idx,
                    'component': comp_name,
                    'point': point_name,
                    'x': round(float(coords[0]), 4),
                    'y': round(float(coords[1]), 4),
                    'z': round(float(coords[2]), 4) if len(coords) > 2 else 0,
                    'confidence': round(float(conf), 4)
                })
    
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False)
    
    print(f"✅ تم الحفظ في: {output_path}")
    print(f"   📊 عدد الصفوف: {len(df):,}")
    
    return df

# مثال: تصدير اليدين فقط
# df = pose_to_csv(pose, "hands_only.csv", 
#                  component_filter=["LEFT_HAND_LANDMARKS", "RIGHT_HAND_LANDMARKS"])

## 6️⃣ دالة تحويل شاملة

In [ ]:
def convert_pose(pose_path: str, output_format: str, output_path: str = None):
    """
    دالة تحويل شاملة
    
    Parameters:
    -----------
    pose_path : str
        مسار ملف الـ .pose
    output_format : str
        الصيغة المطلوبة: 'json', 'npz', 'parquet', 'csv'
    output_path : str, optional
        مسار الملف الناتج (يُولّد تلقائياً إذا لم يُحدد)
    """
    # تحميل الملف
    pose = load_pose_file(pose_path)
    
    # تحديد اسم الملف الناتج
    base_path = Path(pose_path).stem
    
    format_map = {
        'json': ('.json', pose_to_json),
        'npz': ('.npz', pose_to_npz),
        'parquet': ('.parquet', pose_to_parquet),
        'csv': ('.csv', pose_to_csv)
    }
    
    if output_format not in format_map:
        raise ValueError(f"صيغة غير مدعومة: {output_format}. الصيغ المدعومة: {list(format_map.keys())}")
    
    ext, converter = format_map[output_format]
    
    if output_path is None:
        output_path = f"{base_path}{ext}"
    
    return converter(pose, output_path)

# أمثلة على الاستخدام:
# convert_pose("input.pose", "json")
# convert_pose("input.pose", "npz")
# convert_pose("input.pose", "parquet")
# convert_pose("input.pose", "csv")

## 7️⃣ تحويل دفعي (Batch Conversion)

In [ ]:
from pathlib import Path
from tqdm import tqdm

def batch_convert(input_dir: str, output_format: str, output_dir: str = None):
    """
    تحويل مجلد كامل من ملفات .pose
    """
    input_path = Path(input_dir)
    pose_files = list(input_path.glob("*.pose"))
    
    if not pose_files:
        print("⚠️ لم يتم العثور على ملفات .pose")
        return
    
    print(f"📁 تم العثور على {len(pose_files)} ملف")
    
    # إنشاء مجلد الإخراج
    if output_dir is None:
        output_dir = input_path / f"converted_{output_format}"
    else:
        output_dir = Path(output_dir)
    
    output_dir.mkdir(exist_ok=True)
    
    # تحويل الملفات
    for pose_file in tqdm(pose_files, desc="التحويل"):
        try:
            ext = {'json': '.json', 'npz': '.npz', 'parquet': '.parquet', 'csv': '.csv'}[output_format]
            output_path = output_dir / f"{pose_file.stem}{ext}"
            convert_pose(str(pose_file), output_format, str(output_path))
        except Exception as e:
            print(f"❌ خطأ في {pose_file.name}: {e}")
    
    print(f"\n✅ تم التحويل في: {output_dir}")

# batch_convert("pose_files_folder", "parquet")

## 8️⃣ مقارنة أحجام الملفات

In [ ]:
def compare_file_sizes(pose_path: str):
    """
    مقارنة أحجام الملفات بالصيغ المختلفة
    """
    import tempfile
    import os
    
    pose = load_pose_file(pose_path)
    original_size = Path(pose_path).stat().st_size
    
    results = {'pose (original)': original_size}
    
    with tempfile.TemporaryDirectory() as tmpdir:
        # JSON
        json_path = os.path.join(tmpdir, "test.json")
        pose_to_json(pose, json_path)
        results['json'] = Path(json_path).stat().st_size
        
        # NPZ (مضغوط)
        npz_path = os.path.join(tmpdir, "test.npz")
        pose_to_npz(pose, npz_path, compressed=True)
        results['npz (compressed)'] = Path(npz_path).stat().st_size
        
        # Parquet
        parquet_path = os.path.join(tmpdir, "test.parquet")
        pose_to_parquet_efficient(pose, parquet_path)
        results['parquet (efficient)'] = Path(parquet_path).stat().st_size
    
    # عرض النتائج
    print("\n" + "=" * 50)
    print("📊 مقارنة أحجام الملفات")
    print("=" * 50)
    
    for fmt, size in sorted(results.items(), key=lambda x: x[1]):
        ratio = size / original_size * 100
        print(f"{fmt:25} {size/1024:10.2f} KB  ({ratio:6.1f}%)")

# compare_file_sizes("your_file.pose")

## 📚 ملخص الصيغ

| الصيغة | المميزات | العيوب | الاستخدام الأمثل |
|--------|----------|--------|------------------|
| `.pose` | صيغة المكتبة الأصلية، مضغوطة | خاصة بالمكتبة | التخزين والعمل مع المكتبة |
| `.json` | قراءة سهلة، متوافقة مع الويب | حجم كبير | الويب، التصحيح |
| `.npz` | فعالة للـ NumPy، مضغوطة | تحتاج NumPy | التحليل العلمي |
| `.parquet` | فعالة للبيانات الكبيرة | تحتاج pandas | التحليلات الكبيرة |
| `.csv` | متوافقة مع كل البرامج | حجم كبير جداً | Excel، برامج أخرى |

### الخطوة التالية
انتقل للـ Notebook التالي لتعلم كيفية **قراءة الملفات** في Python و JavaScript!